In [ ]:
import random
import statistics
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
from scipy.stats import linregress
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.constants import k as k_B  # Boltzmann constant in J/K
from scipy.constants import G, proton_mass
from functools import partial
# from lab_functions_1 import Ax, vcgrab, Tc, Lambdacalc, dLdTfunc, dVcdrfunc
import lab_functions_1 as lf
import WiersmaCopy as Cool
G = 6.6743*10**-8
mp_g = proton_mass*1e3
k_Bcgs = k_B*1e7
mu = 0.6
gamma = 5/3
ktc = 3.0857e21
etkv = 6.2415*10**8

In [ ]:
# Create an instance of the cooling object
Z2Zsun = 0.3  # solar metallicity
z = 0.6       # redshift
cooling = Cool.Wiersma_Cooling(Z2Zsun=Z2Zsun, z=z)

# Generate random test values for T and nH
n_samples = 5
T_samples = 10**np.random.uniform(4.2, 8.16, size=n_samples)      # T from 1e4 to 1e8 K
nH_samples = 10**np.random.uniform(-3.7, -0.52, size=n_samples)    # nH from 1e-6 to 1e2 cm^-3

# Test LAMBDA
print("T [K]       nH [cm^-3]     Lambda [erg cm^3 / s]")
for T, nH in zip(T_samples, nH_samples):
    Lambda_val = cooling.LAMBDA(T, nH)
    print(f"{T: .2e}   {nH: .2e}     {Lambda_val: .2e}")

In [ ]:
left = np.array([9.3, 9.5, 9.7, 9.9, 10.1, 10.3, 10.5, 10.7, 10.9, 11.1, 11.3, 11.5, 11.7, 11.9, 12.1, 12.3, 12.5, 12.7, 12.9, 
    13.1, 13.3, 13.5, 13.7, 13.9, 14.1, 14.3, 14.5, 14.7])

right = np.array([-4.560623, -4.336346, -4.1121, -3.887918, -3.663862, -3.440012, -3.216311, -2.991903, -2.763624, -2.524845, 
    -2.269454, -2.005525, -1.769973, -1.619732, -1.591587, -1.669076, -1.79781, -1.929445, -2.045038, -2.146987, -2.242711, 
    -2.336809, -2.430866, -2.525143, -2.61958, -2.714102, -2.808667, -2.903251])

f = interp1d(left, right, kind='cubic', fill_value='extrapolate')
print(f(15.08))

plt.plot(left, right, marker='o', linestyle='-', color='b')
plt.xlabel('log10(Halo Mass)')
plt.ylabel('log10(Stellar Mass)')
plt.title('Stellar Mass vs Halo Mass at z ~ 0.59')
plt.grid(True)
plt.show()

In [ ]:
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=.0457)
z = 0.597
rho_c = cosmo.critical_density(z)
rho_m = cosmo.Om(z) * cosmo.critical_density(z)
rho_m = rho_m.value
x = cosmo.Om(z) - 1
delc = 18 * np.pi**2 + 82 * x - 39 * x**2
delm = delc / cosmo.Om(z)
Rvir = (3 * 2e15 * 1.988e33 / (4 * np.pi * delm * rho_m))**(1/3)
print(f"Rvir = {Rvir/ktc} rs = {Rvir/(4.3*ktc)}")

In [ ]:
delcv = 18 * np.pi**2 + 82 * (cosmo.Om(0.004) - 1) - 39 * (cosmo.Om(0.004) - 1)
delmv = delcv / cosmo.Om(0.004)
rho_cv = cosmo.critical_density(0.004)
rho_mv = cosmo.Om(0.004) * rho_cv
Rvirv = ((3 * 6.3e14 * 1.988e33 / (4 * np.pi * delmv * rho_mv))**(1/3)).value
print(((3 * 6.3e14 * 1.988e33 / (4 * np.pi * delmv * rho_mv))**(1/3)).value / ktc)

In [ ]:
R200c = (3 * 2e15 * 1.988e33 / (4 * np.pi * 200 * rho_c))**(1/3)
R200m = (3 * 2e15 * 1.988e33 / (4 * np.pi * 200 * rho_m))**(1/3)
R200cv = (3 * 6.3e14 * 1.988e33 / (4 * np.pi * 200 * rho_cv))**(1/3)
R200mv = (3 * 6.3e14 * 1.988e33 / (4 * np.pi * 200 * rho_mv))**(1/3)
print(f"R200c = {(R200c/ktc).value}, Rvir = {Rvir/ktc} R200m = {(R200m/ktc)}, R200cv = {(R200cv/ktc).value}, Rvirv = {Rvirv / ktc} R200mv = {(R200mv/ktc).value}")
#print(f"rho_c at z=0.004 = {rho_cv.value}, rho_m = {rho_mv.value}")
#print(f"rho_c at z=0.596 = {rho_c.value}, rho_m = {rho_m}")
#print(cosmo.Om(0.004), cosmo.Om(0.597))

In [ ]:
z = 0.597
Ob0 = cosmo.Ob0  # baryon density today
E_z = cosmo.efunc(z)  # E(z) = H(z)/H0

Ob_z = Ob0 * (1 + z)**3 / E_z**2

print(Ob_z)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define your custom color palette
custom_colors = [
   '#ffffff',
   '#0b9a6d',
   '#18453b'
]

# Create a ListedColormap
s_cmap = LinearSegmentedColormap.from_list("custom_gradient", custom_colors)
s_cmap

In [ ]:
y = np.log10(np.array([3324.598, 1522.680, 1553.475, 1297.309, 725.888, 594.173, 374.898, 346.043, 214.010, 193.623, 155.348, 137.762, 127.159, 112.765,
    96.075, 85.199, 78.642, 69.739, 57.085, 51.647, 39.811, 35.304, 33.246, 30.079, 16.830, 14.925, 14.055, 12.716, 9.607, 8.185,
    4.228, 3.008, 2.563, 1.975]))

y_pairs = y.reshape(-1, 2)

# Compute symmetric error for each pair
errors = np.abs(y_pairs[:, 0] - y_pairs[:, 1]) / 2
stdvi = errors**2

print(stdvi)

In [ ]:
T_grid = np.logspace(4.16, 10, 10000)
L_grid = np.log10(np.array([lf.Lambdacalc(T, 1000*ktc, -1, 1) for T in T_grid]))
L_grid2 = np.log10(np.array([lf.Lambdacalc(T, 1000*ktc, 0.3, 1) for T in T_grid]))

plt.scatter(T_grid, L_grid, s=1)
plt.scatter(T_grid, L_grid2, s=1)
plt.xlabel('Temperature T')
plt.ylabel('Lambda Value')
plt.xscale('log')
plt.xlim(10**4.16, 2e8)
#plt.yscale('log')
plt.title('Lambda as a function of T')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Define temperature and radius grids
T_vals = np.linspace(7, 8.3, 500)      # Temperature range [K]
R_vals = np.logspace(0.47, 3.37, 500) * ktc   # Radius range [code units to physical, adjust as needed]

# Create meshgrid for evaluation
T_mesh, R_mesh = np.meshgrid(T_vals, R_vals, indexing='ij')

# Evaluate Lambda for each (T, R) pair
Lambda_grid = np.zeros_like(T_mesh)

for i in range(T_mesh.shape[0]):
    for j in range(T_mesh.shape[1]):
        T = T_mesh[i, j]
        R = R_mesh[i, j]
        Lambda_grid[i, j] = lf.Lambdacalc(T, R, -1, 1)

# Convert to log10 for plotting clarity
log_Lambda = np.log10(Lambda_grid)

# Plotting
plt.figure(figsize=(8, 6))
pcm = plt.pcolormesh(R_vals / ktc, T_vals, log_Lambda, shading='nearest', cmap=s_cmap)
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Radius [in units of ktc]")
plt.ylabel("Temperature [K]")
plt.title("log10(Lambda) as function of Radius and Temperature")
cbar = plt.colorbar(pcm, label=r"$\log_{10}(\Lambda)$")
plt.grid(True, which='both', ls='--', lw=0.5)
plt.tight_layout()
plt.show()

In [ ]:
T_vals = np.linspace(4.16, 8.5, 100)
R_fixed = 100*ktc

Tarray = np.arange(4.20, 8.16 + 0.001, 0.04)

Larray5 = 10**np.array([
-21.6106,-21.4817,-21.5094,-21.5894,-21.6708,-21.7330,-21.7528,-21.7361,-21.6873,-21.6136,
-21.5229,-21.4248,-21.3330,-21.2392,-21.1599,-21.0871,-21.0286,-20.9858,-20.9582,-20.9451,
-20.9602,-20.9861,-21.0079,-21.0052,-20.9968,-20.9834,-20.9707,-20.9670,-20.9647,-20.9722,
-21.0099,-21.0958,-21.2556,-21.4404,-21.5834,-21.6649,-21.7010,-21.7207,-21.7454,-21.7931,
-21.8615,-21.9160,-21.9476,-21.9656,-21.9760,-21.9938,-22.0144,-22.0355,-22.0433,-22.0536,
-22.0725,-22.1082,-22.1698,-22.2486,-22.3375,-22.4159,-22.4820,-22.5278,-22.5612,-22.5811,
-22.5905,-22.5940,-22.5857,-22.5797,-22.5694,-22.5612,-22.5573,-22.5566,-22.5635,-22.5718,
-22.5831,-22.6010,-22.6169,-22.6391,-22.6596,-22.6840,-22.7021,-22.7090,-22.7121,-22.7129,
-22.7135,-22.7065,-22.6959,-22.6840,-22.6722,-22.6574,-22.6418,-22.6337,-22.6173,-22.5998,
-22.5828,-22.5679,-22.5500,-22.5324,-22.5152,-22.4972,-22.4822,-22.4644,-22.4467,-22.4287
])

Lambda_vals = []
for T in T_vals:
    val = lf.Lambdacalc(T, R_fixed, 0.5, 1)
    Lambda_vals.append(val)

Lambda_vals = np.array(Lambda_vals)

# Plot Lambda vs Temperature
plt.figure(figsize=(7,5))
plt.loglog(T_vals, Lambda_vals)
plt.scatter(Tarray, Larray5, s=20, color='red')
plt.xlabel("Temperature [K]")
plt.xlim(4, 8.5)
plt.ylabel(r"$\Lambda$")
plt.title(r"$\Lambda$ vs Temperature (Z = 0.5Z)")
plt.grid(True, which='both', ls='--', lw=0.5)
plt.show()

In [ ]:
T_vals = np.linspace(4.16, 8.5, 100)
R_fixed = 100*ktc

Larray10 = 10**np.array([
-21.6087,-21.4779,-21.5009,-21.5702,-21.6311,-21.6603,-21.6373,-21.5738,-21.4838,-21.3771,
-21.2642,-21.1525,-21.0529,-20.9557,-20.8769,-20.8078,-20.7546,-20.7154,-20.6877,-20.6713,
-20.6828,-20.7056,-20.7242,-20.7180,-20.7069,-20.6912,-20.6768,-20.6720,-20.6687,-20.6755,
-20.7130,-20.7993,-20.9603,-21.1472,-21.2921,-21.3746,-21.4107,-21.4301,-21.4547,-21.5029,
-21.5726,-21.6281,-21.6600,-21.6782,-21.6885,-21.7067,-21.7277,-21.7494,-21.7574,-21.7680,
-21.7877,-21.8248,-21.8893,-21.9727,-22.0680,-22.1536,-22.2272,-22.2797,-22.3195,-22.3434,
-22.3570,-22.3622,-22.3553,-22.3501,-22.3409,-22.3324,-22.3309,-22.3340,-22.3443,-22.3580,
-22.3758,-22.4007,-22.4264,-22.4606,-22.4953,-22.5331,-22.5666,-22.5865,-22.6018,-22.6128,
-22.6215,-22.6209,-22.6164,-22.6101,-22.6019,-22.5907,-22.5782,-22.5715,-22.5577,-22.5421,
-22.5275,-22.5143,-22.4980,-22.4822,-22.4671,-22.4508,-22.4375,-22.4215,-22.4056,-22.3893
])

Lambda_vals = []
for T in T_vals:
    val = lf.Lambdacalc(T, R_fixed, 1, 1)
    Lambda_vals.append(val)

Lambda_vals = np.array(Lambda_vals)

# Plot Lambda vs Temperature
plt.figure(figsize=(7,5))
plt.loglog(T_vals, Lambda_vals)
plt.scatter(Tarray, Larray10, s=20, color='red')
plt.xlabel("Temperature [K]")
plt.xlim(7, 8.5)
plt.ylabel(r"$\Lambda$")
plt.title(r"$\Lambda$ vs Temperature (Z = 0.5Z)")
plt.grid(True, which='both', ls='--', lw=0.5)
plt.show()

In [ ]:
from scipy.optimize import curve_fit
rarray = np.array([13.75, 33.75, 51.25, 76.25, 115.0, 193.75, 352.5])
zarray = np.array([0.76510, 1.04027, 0.61074, 0.63423, 0.43289, 0.24161, 0.03691])

# Model function
def model(x, A, B):
    return A * np.exp(-x * B)

# Initial parameter guess (A, B, C)
initial_guess = [1.0, 0.01]

# Curve fitting
popt, pcov = curve_fit(model, rarray, zarray, p0=initial_guess)

# Extract fitted parameters
A_fit, B_fit = popt
print(f"Fitted parameters:\nA = {A_fit:.5f}, B = {B_fit:.5f}")

# Plotting
x_fit = np.linspace(min(rarray), max(zarray), 300)
y_fit = model(x_fit, *popt)

plt.scatter(rarray, zarray, label="Data", color='red')
plt.plot(x_fit, y_fit, label="Fit: $Ae^{-xB} + C$", color='blue')
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title("Exponential Decay Fit")
plt.grid(True)
plt.show()

In [ ]:
from scipy.interpolate import RegularGridInterpolator
Tarray = np.arange(4.20, 8.16 + 0.001, 0.04)
Lambda3 = np.array([-21.6114, -21.4833, -21.5129, -21.5974, -21.6878, -21.7659, -21.8092, -21.8230, -21.8059, -21.7621, -21.6941, -21.6111, 
    -21.5286, -21.4387, -21.3589, -21.2816, -21.2168, -21.1700, -21.1423, -21.1331, -21.1525, -21.1820, -21.2077, -21.2093, -21.2043, 
    -21.1937, -21.1832, -21.1811, -21.1799, -21.1883, -21.2263, -21.3118, -21.4700, -21.6521, -21.7926, -21.8728, -21.9090, -21.9290, 
    -21.9539, -22.0008, -22.0678, -22.1209, -22.1521, -22.1698, -22.1804, -22.1977, -22.2178, -22.2383, -22.2459, -22.2557, -22.2736, 
    -22.3075, -22.3657, -22.4391, -22.5207, -22.5909, -22.6490, -22.6878, -22.7148, -22.7308, -22.7361, -22.7379, -22.7283, -22.7216, 
    -22.7102, -22.7023, -22.6962, -22.6921, -22.6959, -22.6994, -22.7050, -22.7170, -22.7249, -22.7378, -22.7480, -22.7629, -22.7710, 
    -22.7697, -22.7655, -22.7605, -22.7565, -22.7461, -22.7323, -22.7176, -22.7039, -22.6873, -22.6700, -22.6613, -22.6436, -22.6251,
    -22.6071, -22.5914, -22.5727, -22.5542, -22.5360, -22.5172, -22.5014, -22.4828, -22.4642, -22.4455])

Lambda5 = np.array([-21.6106,-21.4817,-21.5094,-21.5894,-21.6708,-21.7330,-21.7528,-21.7361,-21.6873,-21.6136, -21.5229, -21.4248,
    -21.3330,-21.2392,-21.1599,-21.0871,-21.0286,-20.9858,-20.9582,-20.9451, -20.9602,-20.9861,-21.0079,-21.0052,-20.9968,-20.9834,
    -20.9707,-20.9670,-20.9647,-20.9722,-21.0099,-21.0958,-21.2556,-21.4404,-21.5834,-21.6649,-21.7010,-21.7207,-21.7454,-21.7931,
    -21.8615,-21.9160,-21.9476,-21.9656,-21.9760,-21.9938,-22.0144,-22.0355,-22.0433,-22.0536,-22.0725,-22.1082,-22.1698,-22.2486,
    -22.3375,-22.4159,-22.4820,-22.5278,-22.5612,-22.5811,-22.5905,-22.5940,-22.5857,-22.5797,-22.5694,-22.5612,-22.5573,-22.5566,
    -22.5635,-22.5718,-22.5831,-22.6010,-22.6169,-22.6391,-22.6596,-22.6840,-22.7021,-22.7090,-22.7121,-22.7129,-22.7135,-22.7065,
    -22.6959,-22.6840,-22.6722,-22.6574,-22.6418,-22.6337,-22.6173,-22.5998,-22.5828,-22.5679,-22.5500,-22.5324,-22.5152,-22.4972,
    -22.4822,-22.4644,-22.4467,-22.4287])

Lambda10 = np.array([-21.6087,-21.4779,-21.5009,-21.5702,-21.6311,-21.6603,-21.6373,-21.5738,-21.4838,-21.3771, -21.2642,-21.1525,
-21.0529,-20.9557,-20.8769,-20.8078,-20.7546,-20.7154,-20.6877,-20.6713,-20.6828,-20.7056,-20.7242,-20.7180,-20.7069,-20.6912,
-20.6768,-20.6720,-20.6687,-20.6755,-20.7130,-20.7993,-20.9603,-21.1472,-21.2921,-21.3746,-21.4107,-21.4301,-21.4547,-21.5029,
-21.5726,-21.6281,-21.6600,-21.6782,-21.6885,-21.7067,-21.7277,-21.7494,-21.7574,-21.7680,-21.7877,-21.8248,-21.8893,-21.9727,
-22.0680,-22.1536,-22.2272,-22.2797,-22.3195,-22.3434,-22.3570,-22.3622,-22.3553,-22.3501,-22.3409,-22.3324,-22.3309,-22.3340,
-22.3443,-22.3580,-22.3758,-22.4007,-22.4264,-22.4606,-22.4953,-22.5331,-22.5666,-22.5865,-22.6018,-22.6128,-22.6215,-22.6209,
-22.6164,-22.6101,-22.6019,-22.5907,-22.5782,-22.5715,-22.5577,-22.5421,-22.5275,-22.5143,-22.4980,-22.4822,-22.4671,-22.4508,
-22.4375,-22.4215,-22.4056,-22.3893])

Z_values = np.array([0.3, 0.5, 1.0])


logLambda_grid = np.vstack([Lambda3, Lambda5, Lambda10])

interpolator = RegularGridInterpolator((Z_values, Tarray), logLambda_grid, bounds_error=False, fill_value=None)

Z_fine = np.linspace(0.15, 1.0, 250)
T_fine = np.linspace(6.8, 8.4, 250)
ZZ, TT = np.meshgrid(Z_fine, T_fine, indexing='ij')

pts = np.stack([ZZ.ravel(), TT.ravel()], axis=-1)
logLambda_fine = interpolator(pts).reshape(ZZ.shape)

plt.figure(figsize=(7,5))
plt.contourf(TT, ZZ, logLambda_fine, levels=42, cmap=s_cmap)
plt.xlabel('log T')
plt.ylabel('Z')
plt.colorbar(label='log Λ')
plt.show()

In [ ]:
R = np.logspace(-3, 4, 100)
for Rh in [1, 10, 100]:
    plt.plot(R, 1/((2/np.pi) * np.arctan(R/(2*Rh))))
plt.xscale('log')
plt.yscale('log')